# Libraries

In [14]:
%load_ext autoreload
%autoreload 2

import numpy as np
from omegaconf import OmegaConf
from src.models import models
from src.models import model as mainmodel
from src.models import modelConv2d
from src.models import modelineal
from src.models import decoders
from src import loss_func
from src import train
from src import loader
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from src import optimizer_Factory as of
import Data.genData as genData
from src import custom_plots as cp
from src import Visual_utils as vu
import torchvision
import wandb
import random
import os
import cv2
import glob
import csv

import pandas as pd

from scipy.signal import savgol_filter


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:2000"
torch.cuda.empty_cache() 
torch.manual_seed(42)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Functions

In [31]:
# Function to iterate over folders and process video files
def iterate_and_process_videos(base_path):
    for root, dirs, files in os.walk(base_path):
        # If no subdirectories, we are in a terminal folder
        if not dirs:
            # Iterate over mp4 files that don't contain 'mask' in the filename
            mp4_files = [f for f in files if f.endswith('.mp4') and 'mask' not in f]
            for mp4_file in mp4_files:
                video_path = os.path.join(root, mp4_file)
                numpy_save_path = os.path.join(root, f"{os.path.basename(root)}.npy")
                video_array = process_video(video_path, new_width=100, new_height=56)
                np.save(numpy_save_path, video_array)

# Function to process video: resize, convert to grayscale, and normalize
def process_video(video_path, new_width, new_height):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return np.array([])
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize the frame
        resized_frame = cv2.resize(frame, (new_width, new_height))
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        # Normalize the frame to range [0, 1]
        normalized_frame = gray_frame / 255.0
        frames.append(normalized_frame)
    
    cap.release()
    
    # Convert list of frames to numpy array
    video_array = np.array(frames)
    return video_array

# Function to calculate mean and std of alpha and beta based on folder1 and folder2
def calculate_mean_std(base_path):
    csv_file_path = os.path.join(base_path, "experiment_results.csv")
    if not os.path.exists(csv_file_path):
        print(f"Error: CSV file {csv_file_path} not found.")
        return
    
    # Read CSV into DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Group by folder1 and folder2, and calculate mean and std for alpha and beta
    grouped = df.groupby(['folder1', 'folder2']).agg(
        alpha_mean=('alpha', 'mean'),
        alpha_std=('alpha', 'std'),
        beta_mean=('beta', 'mean'),
        beta_std=('beta', 'std')
    ).reset_index()
    
    # Save the results to a new CSV file
    output_csv_path = os.path.join(base_path, "mean_std_results.csv")
    grouped.to_csv(output_csv_path, index=False)
    print(f"Mean and std results saved to {output_csv_path}")

# Function to execute experiment for a particular file and update CSV
def execute_experiment_for_file(base_path, folder1, folder2, folder):
    csv_file_path = os.path.join(base_path, "experiment_results.csv")
    if not os.path.exists(csv_file_path):
        print(f"Error: CSV file {csv_file_path} not found.")
        return
    
    # Read CSV into DataFrame
    df = pd.read_csv(csv_file_path)

    print(f"Executing experiment for folder1={folder1}, folder2={folder2}, folder={folder}")

    #print(df)

    # Get the path of the numpy file
    npy_file_path = os.path.join(base_path, folder1, folder2, folder, f"{folder}.npy")

    print(f"Processing file {npy_file_path}")
    
    # Find the row that matches folder1, folder2, and folder
    matching_index = df[(df['folder1'] == folder1) & (df['folder2'] == folder2) & (df['folder'] == int(folder) )].index
    if matching_index.empty:
        print(f"Error: No matching row found for folder1={folder1}, folder2={folder2}, folder={folder}")
        return
    
    # Get the path of the numpy file
    npy_file_path = os.path.join(base_path, folder1, folder2, folder, f"{folder}.npy")
    if not os.path.exists(npy_file_path):
        print(f"Error: Numpy file {npy_file_path} not found.")
        return
    
    # Execute experiment function
    alpha, beta = test_execute_experiment(npy_file_path, dynamics=folder1, dt=0.1)
    
    # Update the row in the DataFrame
    df.at[matching_index[0], 'alpha'] = alpha
    df.at[matching_index[0], 'beta'] = beta
    
    # Save the updated DataFrame back to CSV
    df.to_csv(csv_file_path, index=False)
    print(f"Updated row for folder1={folder1}, folder2={folder2}, folder={folder}")

In [4]:
base_path = "./Data/Real_videos"  # Change to your base folder path
iterate_and_process_videos(base_path)

# Model

In [7]:
def execute_experiment(path, dynamics, dt):

    torch.cuda.empty_cache() 
    torch.manual_seed(42)


    data_folder = np.load(path, allow_pickle=True)
    data_train = data_folder
    # print max
    print(np.max(data_train))
    print(data_train.shape)

    #dt = 3/30
    train_dataloader, test_dataloader, train_x, val_x  = loader.getLoader_folder(data_train, split=True)



    latentEncoder_I = mainmodel.EndPhys(dt = dt,
                                    pmodel = dynamics,
                                    init_phys = 1.0, 
                                    initw=True)

    latentEncoder_I, log, params  = train.train(latentEncoder_I, 
                                    train_dataloader, 
                                    test_dataloader,
                                    init_phys = 0.1,                                 
                                    loss_name='latent_loss')
def test_execute_experiment(path, dynamics, dt):
    # This is a placeholder implementation. Replace with the actual logic.
    alpha = np.random.rand()  # Random value as a placeholder
    beta = np.random.rand()   # Random value as a placeholder
    return alpha, beta
  
def iterate_and_execute_experiment(base_path):
    csv_file_path = os.path.join(base_path, "experiment_results.csv")
    with open(csv_file_path, mode='w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(["folder1", "folder2", "folder", "alpha", "beta"])
        
        for root, dirs, files in os.walk(base_path):
            # Iterate over .npy files
            npy_files = [f for f in files if f.endswith('.npy')]
            for npy_file in npy_files:
                npy_file_path = os.path.join(root, npy_file)
                folder_hierarchy = root.split(os.sep)[-3:]
                folder1 = folder_hierarchy[0] if len(folder_hierarchy) > 0 else ""
                folder2 = folder_hierarchy[1] if len(folder_hierarchy) > 1 else ""
                folder = folder_hierarchy[2] if len(folder_hierarchy) > 2 else ""
                
                # Execute experiment function
                alpha, beta = test_execute_experiment(npy_file_path, dynamics=folder1, dt=0.1)
                
                # Write results to CSV
                csv_writer.writerow([folder1, folder2, folder, alpha, beta])

In [15]:
base_path = "./Data/Real_videos"  # Change to your base folder path
iterate_and_execute_experiment(base_path)
calculate_mean_std(base_path)

Mean and std results saved to ./Data/Real_videos\mean_std_results.csv


In [33]:

folder1 = "bouncing_ball"
folder2 = "table"
folder = "01"
execute_experiment_for_file(base_path, folder1, folder2, folder)

Executing experiment for folder1=bouncing_ball, folder2=table, folder=01
Processing file ./Data/Real_videos\bouncing_ball\table\01\01.npy
Updated row for folder1=bouncing_ball, folder2=table, folder=01
